In [ ]:
import numpy as np
import math as m
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from yahoo_fin import options
from yahoo_fin.stock_info import get_data
from dateutil import parser
from datetime import datetime
from pandas_datareader.data import DataReader as dr
%matplotlib inline

In [ ]:
# Black-Scholes formula
def BSF(S, E, T, sigma, r):
    
    # where:
    # S is the initial value of the underlying asset
    # E is the stike price
    # r is the risk free rate
    # sigma is the volatility of the underlying asset
    # T is the time to maturity
    # D is the dividend yield
    # option is to define whether a call or put option is being valued
    
    # d1 is a variable in the black-scholes formula
    d1 = (np.log(S / E) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    
    # d2 is a variable in the black-scholes formula
    d2 = (np.log(S / E) + (r - 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))

    value = S * 0.5 * (1 + m.erf(d1 / np.sqrt(2))) - \
        (E * np.exp(-r * T) * 0.5 * (1 + m.erf(d2 / np.sqrt(2))))
        
    
    return value

In [ ]:
# the derivative of the Black-Scholes Call price w.r.t volatility
def Vega(S, E, T, sigma, r):
    
    # d1 is a variable in the black-scholes formula
    d1 = (np.log(S / E) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T)
    
    return vega

In [ ]:
# Bisection method for calculating implied volatility
def bisection(S, E, T, r, tol, MP, sigma):
    # MP is options market price
    upper_vol = 4 # upper estimate
    lower_vol = 0.005 # lower estimate
    iteration = 0 # iteration counter
    tol = 1e-8 # degree of accuracy
    eps = 1 # placeholder for error between estimates 
    max_iter = 1000 # max iteration limit (avoid infinte loop)
    BS_Low = BSF(S=S, E=E, T=T, sigma=lower_vol, r=r) # BS price for low estimate
    BS_High = BSF(S=S, E=E, T=T, sigma=upper_vol, r=r) # BS price for high estimat
    vi = lower_vol + (MP - BS_Low)*(upper_vol - lower_vol)/(BS_High - BS_Low) # Bisection formula
    tempVal = BSF(S=S, E=E, T=T, sigma=vi, r=r) # BS price for latest estimate
    while eps > tol:
        iteration += 1
        if iteration >= max_iter:     # the count will be used to break an infinte loop from occuring
            print('break E={}, T={}, MP={}, S={}, E+MP-S={}'.format(E, T, MP, S, E+MP-S))
            break
        if tempVal < MP:
            lower_vol = vi
        else:
            upper_vol = vi
            
        BS_Low = BSF(S=S, E=E, T=T, sigma=lower_vol, r=r)
        BS_High = BSF(S=S, E=E, T=T, sigma=upper_vol, r=r)
        vi = lower_vol + (MP - BS_Low)*(upper_vol - lower_vol)/(BS_High - BS_Low)
        tempVal = BSF(S=S, E=E, T=T, sigma=vi, r=r)
        eps = abs(MP - tempVal)
    return vi

In [ ]:
# Newton-Raphson method for calculating implied volatility (IV)
def NewtonRaphson(MP, S1, E1, r1, T1):
    epsilon = 1 # constant to be used in the calculation
    tol = 1e-5 # tolerance of volatility estimate
    count = 0 # count how many estimates are required 
    max_iter = 10000 # cut the loop once max_iter has been reached
    sigma = np.sqrt(np.abs(np.log(S1/E1) + r1*T1)*(2/T1)) # initial estimate of volatility
    while epsilon > tol:
        count += 1                # Incase there is an error in the calculation, 
        if count >= max_iter:     # the count will be used to break an infinte loop from occuring
            print('break E={}, T={}, MP={}, S={}, E+MP-S={}'.format(E1, T1, MP, S1, E1+MP-S1))
            break
        orig_vol = sigma # store the previous estimate
        BSP = BSF(S=S1, E=E1, T=T1, sigma=sigma, r=r1) # Black-Scholes price from previous estimate
        vega = Vega(S=S1, E=E1, T=T1, sigma=sigma, r=r1)
        sigma = sigma - (BSP - MP) / vega # Newton-Raphson calculation
        epsilon = abs(sigma - orig_vol) # Tolerance to be obtained in the estimate to break the calultion loop
    if m.isnan(sigma)==True or sigma < 0:
        sigma = bisection(S=S1, E=E1, T=T1, r=r1, tol=tol, MP=MP, sigma=sigma)
    return sigma

In [ ]:
ticker = 'MSFT' # set stock to be analysed, e.g. MSFT = Microsoft Corporation
stock = options.get_calls(ticker) # Pull information from Yahoo Finance

In [ ]:
today = datetime.today().date() # set todays date
Maturity = options.get_expiration_dates(ticker) # pull the time to maturities for the option
for i in range(len(Maturity)):
    Maturity[i] = Maturity[i].replace(',', '') # format them so python can subtract from todays date

In [ ]:
S = get_data(ticker).iloc[-1,3] # Latest asset price

In [ ]:
# Using the US treasury and Libor rate, create the term struncture for the risk-free rate
syms = ['DGS1MO', 'DGS3MO', 'DGS6MO', 'DGS1', 'DGS2', 'DGS3'] # US treasury tickers
yc = dr(syms, 'fred') # Pull from the federal reserve website
names = dict(zip(syms, ['1m', '3m', '6m', '1yr', '2yr', '3yr'])) # maturity lenght for the rates
yc = yc.rename(columns=names)
yc = yc[['1m', '3m', '6m','1yr', '2yr', '3yr']]
USr = yc.iloc[-1,:].to_numpy()/100 # US treasury constant maturity rate
# libor rates effective from the 28/05/2021
rWeek = dr('USD1WKD156N', 'fred').iloc[-1]/100 # 1 week libor rate

time = np.array([1/52, 1/12, 1/4, 1/2, 1, 2, 3]) # time i.e. 1 week = 1/52, 1 month = 1/12
r = np.insert(USr, 0, rWeek) # combine US treasury rate and Libor rate

In [ ]:
# Process maturity dates to get in terms of 1 year 
maturityDates = np.zeros(len(Maturity))
for i in range(len(Maturity)):
    t = parser.parse(Maturity[i]).date() - today
    maturityDates[i] = t.days/365
maturityDates

In [ ]:
# Choose the option maturiyt rate and calculate the risk-free for that maturity
# Change the 10 in square brackets to get a different maturity date
opt = options.get_calls(ticker, Maturity[10])
newMat = parser.parse(Maturity[10]).date() - today
newMat = newMat.days/365
timeNew = np.sort(np.append(time, newMat))
rNew = np.interp(timeNew, time, r) # interpolate interest rates to find the interest rate for our maturity date
rOption = rNew[np.where(timeNew == newMat)]
rOption

In [ ]:
CallPrice = opt.iloc[:,3] # list of option prices for each strike
strike = opt.iloc[:,2] # stike prices available for the maturity date choosen

In [ ]:
sigmaImplied = np.zeros(len(CallPrice)) # place holder for implied volatility 
for i in range(len(CallPrice)):
    sigmaImplied[i] = NewtonRaphson(MP = CallPrice[i], S1=S, E1=strike[i], r1=rOption, T1=newMat)
sigmaImplied[16] = 0
# Remove bad date from Yahoo
ind1 = np.where(~np.isnan(sigmaImplied)) # remove NA's
ind2 = np.where(~np.isinf(sigmaImplied[np.asarray(ind1)[0,:]])) # Remove infinity volatilities
ind = np.where(np.round(sigmaImplied[np.asarray(ind2)[0,:]], 1) !=0.) # Remove very small volatilities
sigmaImplied = pd.Series(sigmaImplied)

In [ ]:
callData = pd.concat([strike.iloc[ind], CallPrice.iloc[ind], sigmaImplied.iloc[ind]], axis=1)

In [ ]:
# Plot reseults
plt.figure(figsize=(10,7.5))
plt.plot(callData.iloc[1:,0], callData.iloc[1:,2], label='BS IV')
plt.xlabel('Strike Price', fontsize=14)
plt.ylabel(' Implied Volatility %', fontsize=14)